# Assignment 2 - Predictive Process Monitoring

*Due: Friday, 15 December, 2023 at 14:00 CET*

In this assignment, you will learn to use several regression models to predict the process remaining time. In addition, you will also show that you can evaluate their performance and discuss the results in a report. The learning objectives of this assignment are: 

- use the data aggregation, feature encoding, and data transformation techniques to preprocess event data
- use the regression models to predict the remaining time of ongoing cases. 
- perform cross validation and fine-tune the model parameters of each algorithm
- calculate model performance (e.g., MAE, MSE, RMSE, R^2, etc.)
- design experiments to compare the performance of algorithms
- reflect on the difference between different models


This assignment includes two algorithms: Regression Tree (or Random Forest Regression) and kNN regressor. Following a similar structure as the first assignment, your first task is to perform data exploration and data cleaning. 
In Task 2, you will perform two trace encoding techniques (covered during Lecture 07). 
In Task 3-4, you will use the two algorithms to learn regression models to forecast the remaining time of each case after each event. 
In Task 5, you will compare the algorithms and evaluate their results. 

Please note that Task 3 and 4 have the following structure:
1. First, find the library (e.g., sklearn examples) and try out the algorithm by simply training the model on the training data (do not consider any parameters or cross validation just yet); 
2. Train the model with the training data by using cross validation and find the best parameter setting for the parameters of interest;
3. Report the average MAE, MSE, RMSE, and R^2 of all validation sets;
4. Finally, test the optimal model that has the best fitting parameters on your held-out test data, and report its MAE, MSE, RMSE, and R^2. 

Note that, in Task 5, you will need all the calculated MAE, MSE, RMSE, and R^2 on both encoded data from previous tasks. Make sure you save these to a list or dictionary so you can easily evaluate and compare the results. 



## Task 1: Exploring the data set



### Data set: Sepsis

Import the file *sepsis.csv* to load the Sepsis data set. This real-life event log contains events of sepsis cases from a hospital. Sepsis is a life threatening condition typically caused by an infection. One case represents a patient's pathway through the treatment process. The events were recorded by the ERP (Enterprise Resource Planning) system of the hospital. The original data set contains about 1000 cases with in total 15,000 events that were recorded for 16 different activities. Moreover, 39 data attributes are recorded, e.g., the group responsible for the activity, the results of tests and information from checklists. 

Additional information about the data can be found :
- https://data.4tu.nl/articles/dataset/Sepsis_Cases_-_Event_Log/12707639
- http://ceur-ws.org/Vol-1859/bpmds-08-paper.pdf




In [1]:
import pandas as pd
import numpy as np

In [2]:
data_sepsis = pd.read_csv("./sepsis.csv", sep=";")

In [ ]:
data_sepsis.info()

In [ ]:
data_sepsis.describe()

In [ ]:
data_sepsis.head()

In [ ]:
len(data_sepsis['Case ID'].unique())


### 1.1 Exploratory data analysis

For the data set, create 2-3 figures and tables that help you understand the data 

**Use the column "remtime" (which indicates the remaining time of each case after each corresponding event) as the response variable for regression**

Note that some of these variables are categorical variables. How would you preprocess these variables?


#### Tips: ---------------

During the data exploration, you, as a team, are trying to get an impression of the data. You will create figures and/or tables that help you to get to know the data. While exploring the data, you may also consider answering the following questions, which may help you understand the data better. For example, 

- How many variables are in the data? What are the data type and the distribution of each variable? 
- What is the discribution of the response variable?
- Are the variables informative?
- Are any pair of the potential predictor variables highly correlated?
- (Should the variables be normalized or not?)
- (Any relevant, useful preprocessing steps that may be taken?)



Make sure to at least check the data type of each variable and to understand the distribution of each variable, especially the response variable. 

Try to find out what factors seem to determine whether an instance is an outlier or not. What do you conclude?

*For creating data visualizations, you may consider using the matplot library and visit the [matplot gallery](https://matplotlib.org/stable/gallery/index.html) for inspiration (e.g., histograms for distribution, or heatmaps for feature correlation).*

In [ ]:
# import packages
import matplotlib.pyplot as plt


# TODO: plot figure(s)


### 1.2 Data cleaning

You have now gathered some information about the data during the data exploration task. You also know from the assignment description that you will be using regression trees and kNN regression models to predict the remaining time.

Based on the above information, decide on which cleaning steps you will need to perform and implement them accordingly.


In [3]:
def object_to_bool(df_row: pd.Series):
    df_row.replace({'True': 1, 'False': 0, 'missing': np.nan}, inplace=True)

In [4]:
for column in data_sepsis:
    object_to_bool(data_sepsis[column])

In [5]:
data_sepsis['Complete Timestamp'] = pd.to_datetime(data_sepsis['Complete Timestamp'])

In [6]:
df_sorted = data_sepsis.sort_values(by="Complete Timestamp")
df_sorted.reset_index(inplace=True, drop=True)

In [9]:
pd.set_option("display.max_columns", None)

In [10]:
df_sorted.groupby(by="Case ID")

In [11]:
df_sorted.head()

,DiagnosticArtAstrup,DiagnosticBlood,DiagnosticECG,DiagnosticIC,DiagnosticLacticAcid,DiagnosticLiquor,DiagnosticOther,DiagnosticSputum,DiagnosticUrinaryCulture,DiagnosticUrinarySediment,DiagnosticXthorax,DisfuncOrg,Hypotensie,Hypoxie,InfectionSuspected,Infusion,Oligurie,SIRSCritHeartRate,SIRSCritLeucos,SIRSCritTachypnea,SIRSCritTemperature,SIRSCriteria2OrMore,Age,Case ID,Activity,Diagnose,org:group,CRP,LacticAcid,Leucocytes,Complete Timestamp,duration,month,weekday,hour,remtime,elapsed
0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,90.0,XJ,ER Registration,C,A,0.0,0.0,0.0,2013-11-07 17:18:29,0.000000,11,3,17,1951.0,0.0
1,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,90.0,XJ,ER Triage,C,C,0.0,0.0,0.0,2013-11-07 17:29:18,10.816667,11,3,17,1302.0,649.0
2,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,90.0,XJ,ER Sepsis Triage,C,A,0.0,0.0,0.0,2013-11-07 17:37:32,8.233333,11,3,17,808.0,1143.0
3,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,90.0,XJ,Leucocytes,C,B,0.0,0.0,296.2,2013-11-07 17:51:00,0.000000,11,3,17,0.0,1951.0
4,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,90.0,XJ,CRP,C,B,16.0,0.0,296.2,2013-11-07 17:51:00,13.466667,11,3,17,0.0,1951.0


In [12]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OrdinalEncoder

In [13]:
scaler = MinMaxScaler()
encoder = OrdinalEncoder()

In [15]:
df_analysis = df_sorted.copy()

In [16]:
df_encoded = pd.get_dummies(df_analysis[['Diagnose', 'org:group']], prefix=['Diagnose=', 'org:group='])
for col in df_encoded.columns:
    df_encoded[col].replace({True: 1, False: 0}, inplace=True)
df_analysis = pd.concat([df_analysis, df_encoded], axis=1)

In [17]:
df_analysis['Complete Timestamp'] = encoder.fit_transform(df_analysis['Complete Timestamp'].values.reshape(-1, 1))


In [18]:
scale_cols = ['CRP', 'LacticAcid', 'Leucocytes', 'duration', 'month', 'weekday', 'hour', 'remtime', 'elapsed', 'Complete Timestamp']
df_analysis[scale_cols] = scaler.fit_transform(df_analysis[scale_cols])

In [26]:
df_analysis.drop(columns=['Diagnose', 'org:group'], inplace=True)

In [27]:
df_analysis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5176 entries, 0 to 5175
Data columns (total 95 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   DiagnosticArtAstrup        5107 non-null   float64
 1   DiagnosticBlood            5107 non-null   float64
 2   DiagnosticECG              5107 non-null   float64
 3   DiagnosticIC               5107 non-null   float64
 4   DiagnosticLacticAcid       5107 non-null   float64
 5   DiagnosticLiquor           5107 non-null   float64
 6   DiagnosticOther            5107 non-null   float64
 7   DiagnosticSputum           5107 non-null   float64
 8   DiagnosticUrinaryCulture   5107 non-null   float64
 9   DiagnosticUrinarySediment  5107 non-null   float64
 10  DiagnosticXthorax          5107 non-null   float64
 11  DisfuncOrg                 5107 non-null   float64
 12  Hypotensie                 5107 non-null   float64
 13  Hypoxie                    5107 non-null   float

In [21]:
df_analysis.describe()

,DiagnosticArtAstrup,DiagnosticBlood,DiagnosticECG,DiagnosticIC,DiagnosticLacticAcid,DiagnosticLiquor,DiagnosticOther,DiagnosticSputum,DiagnosticUrinaryCulture,DiagnosticUrinarySediment,DiagnosticXthorax,DisfuncOrg,Hypotensie,Hypoxie,InfectionSuspected,Infusion,Oligurie,SIRSCritHeartRate,SIRSCritLeucos,SIRSCritTachypnea,SIRSCritTemperature,SIRSCriteria2OrMore,Age,CRP,LacticAcid,Leucocytes,Complete Timestamp,duration,month,weekday,hour,remtime,elapsed,Diagnose=_A,Diagnose=_AA,Diagnose=_AB,Diagnose=_AC,Diagnose=_B,Diagnose=_BA,Diagnose=_BB,Diagnose=_C,Diagnose=_CA,Diagnose=_D,Diagnose=_E,Diagnose=_EA,Diagnose=_EB,Diagnose=_ED,Diagnose=_F,Diagnose=_FA,Diagnose=_G,Diagnose=_GA,Diagnose=_GB,Diagnose=_H,Diagnose=_HA,Diagnose=_HD,Diagnose=_IC,Diagnose=_J,Diagnose=_JC,Diagnose=_K,Diagnose=_KA,Diagnose=_LA,Diagnose=_LC,Diagnose=_M,Diagnose=_MA,Diagnose=_MD,Diagnose=_N,Diagnose=_OB,Diagnose=_P,Diagnose=_PC,Diagnose=_Q,Diagnose=_R,Diagnose=_RB,Diagnose=_RC,Diagnose=_S,Diagnose=_SB,Diagnose=_T,Diagnose=_TB,Diagnose=_U,Diagnose=_UA,Diagnose=_UC,Diagnose=_VA,Diagnose=_VB,Diagnose=_W,Diagnose=_XA,Diagnose=_Y,Diagnose=_YA,Diagnose=_Z,Diagnose=_ZA,Diagnose=_other,org:group=_A,org:group=_B,org:group=_C,org:group=_L
count,5107.000000,5107.000000,5107.000000,5107.000000,5107.000000,5107.000000,5107.000000,5107.000000,5107.000000,5107.000000,5107.000000,5107.000000,5107.000000,5107.000000,5107.000000,5107.000000,5107.000000,5107.000000,5107.000000,5107.000000,5107.000000,5107.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000,5176.000000
mean,0.284120,0.787155,0.726454,0.811044,0.766791,0.005874,0.007832,0.028784,0.450362,0.493832,0.748385,0.061876,0.047778,0.018602,0.810456,0.755238,0.021343,0.776581,0.045819,0.567065,0.768357,0.815743,70.191267,0.059067,0.049752,0.015868,0.499968,0.010563,0.502880,0.488472,0.488887,0.024978,0.032943,0.001932,0.008887,0.002705,0.002318,0.078246,0.002705,0.007728,0.140649,0.009274,0.023377,0.066461,0.002318,0.002125,0.002125,0.007342,0.001932,0.038447,0.004057,0.004830,0.046754,0.003671,0.002318,0.004444,0.002318,0.004637,0.022411,0.007535,0.006762,0.002705,0.009080,0.003284,0.003478,0.009853,0.003478,0.005410,0.002125,0.010240,0.020093,0.003478,0.005216,0.011978,0.002318,0.004250,0.002898,0.009274,0.002512,0.002318,0.003478,0.004637,0.006376,0.002125,0.011012,0.002318,0.009467,0.004637,0.091190,0.366692,0.414219,0.194359,0.024730
std,0.451038,0.409359,0.445822,0.391512,0.422916,0.076426,0.088162,0.167215,0.497579,0.500011,0.433984,0.240954,0.213316,0.135128,0.391978,0.429988,0.144540,0.416578,0.209114,0.495530,0.421923,0.387732,17.418468,0.130418,0.107929,0.035756,0.288220,0.045948,0.324378,0.336341,0.356658,0.072598,0.085516,0.043916,0.093861,0.051942,0.048098,0.268584,0.051942,0.087577,0.347693,0.095861,0.151113,0.249110,0.048098,0.046055,0.046055,0.085376,0.043916,0.192291,0.063573,0.069337,0.211132,0.060482,0.048098,0.066518,0.048098,0.067942,0.148031,0.086484,0.081961,0.051942,0.094867,0.057221,0.058874,0.098782,0.058874,0.073358,0.046055,0.100681,0.140331,0.058874,0.072043,0.108799,0.048098,0.065063,0.053760,0.095861,0.050058,0.048098,0.058874,0.067942,0.079600,0.046055,0.104370,0.048098,0.096845,0.067942,0.287907,0.481948,0.4

In [24]:
df_analysis.isna().sum()

DiagnosticArtAstrup     69
DiagnosticBlood         69
DiagnosticECG           69
DiagnosticIC            69
DiagnosticLacticAcid    69
                        ..
Diagnose=_other          0
org:group=_A             0
org:group=_B             0
org:group=_C             0
org:group=_L             0
Length: 97, dtype: int64

In [ ]:
# Activity to be encoded in task 2
# Case ID is not dropped so it can be used for visualizations and grouping. It is dropped before handing data to models

### 1.3 Process Discovery and Visualization (Optional)

This is an optional task to show you how process discovery and visualizaion can be deployed using the pm4py library. 

(*The following code requires the graphviz library to be installed. If you have issues with installing the graphviz, you may try to follow the instructions on Install GraphViz on the [pm4py](https://pm4py.fit.fraunhofer.de/install-page) install page*)

The following code:
- fill in the columns for case id, activity, and timestamps
- convert the data set into an event log
- discover a Directly-follows graph (DFG) and a process model for each event log. 
- you may use the discovered process model in your report



In [ ]:
# Install pm4py library and graphviz library using pip install 
# !pip install -U pm4py
# !pip install graphviz
import pm4py

# Fill in the column names of case id, activity, and time stamps
column_Sepsis_CaseID = 'Case ID' 
column_Sepsis_Activity = 'Activity'
column_Sepsis_Timestamps = 'Complete Timestamp'

data_Sepsis[column_Sepsis_CaseID] = data_Sepsis[column_Sepsis_CaseID].astype(str)


# Convert the data to an event log
log_Sepsis = pm4py.format_dataframe(data_Sepsis, case_id=column_Sepsis_CaseID, activity_key=column_Sepsis_Activity, timestamp_key=column_Sepsis_Timestamps)

# Set the log to be the one that you are interested
log = log_Sepsis

# Create a Directly-Follows Graph (DFG) and plot this graph
dfg, start_activities, end_activities = pm4py.discover_dfg(log)
pm4py.view_dfg(dfg, start_activities, end_activities)

# Discover a Process Model using Inductive Miner and plot this BPMN model
process_tree = pm4py.discover_tree_inductive(log)
bpmn_model = pm4py.convert_to_bpmn(process_tree)
pm4py.view_bpmn(bpmn_model)


## Task 2: Preprocessing and Trace Encoding


### 2.1 Trace Encoding


- Implement the last-2-state encoding for the data set 
- Implement the aggregated encoding for the data set (for example, see [1], Table 6)


<span style="color:gray">[1] Ilya Verenich, Marlon Dumas, Marcello La Rosa, Fabrizio Maria Maggi, Irene Teinemaa:
Survey and Cross-benchmark Comparison of Remaining Time Prediction Methods in Business Process Monitoring. ACM Trans. Intell. Syst. Technol. 10(4): 34:1-34:34 (2019) [Section 1, 2, 4.1, 4.3, 4.6, 5.2, 5.3, 5.4, and 6] </span>

These two encodings are discussed during lecture 7.
In case you find difficult to implement the algorithms, you may also consider use the pandas functions to help you:
- for the last-2-state encoding, check the pandas groupby.DataFrameGroupBy.shift and see the [answer on the stake overflow](https://stackoverflow.com/questions/53335567/use-pandas-shift-within-a-group)
- for the aggregated encoding check the pandas groupby.DataFrameGroupBy and cumsum function and read the [examples and answers on the stake overflow](https://stackoverflow.com/a/49578219)

In [ ]:

def last_n_state_encoding(_data, columnCase, columnActivity, n_state=2):
    new_log = _data['prev_activity'] = np.where(_data[columnCase] == _data[columnCase].shift(n_state),
                                                df_sorted[columnActivity].shift(n_state),
                                                None)
    return(new_log)


# TODO: Implement the function that returns the aggregated state encoding of a log
def agg_state_encoding(_data, columnCase, columnActivity):
    _data.groupby(by=columnCase)
    activity_one_hot = pd.get_dummies(_data[[columnActivity]], prefix='act', columns=[columnActivity])
    return(activity_one_hot)


# TODO: for each of the two data sets, create a last_2_state encoding and an aggregated state encoding
#data_Sepsis_ls = last_n_state_encoding(...)
data_sepsis_ag = agg_state_encoding(df_sorted, 'Case ID', 'Activity')


In [ ]:
data_sepsis_ag

In [ ]:
def agg_state_encoding(_data, columnCase, columnActivity):
    # Group by the specified columnCase
    grouped_data = _data.groupby(by=columnCase)
    
    # Aggregate the one-hot encoded values for columnActivity
    aggregated_data = pd.get_dummies(grouped_data[columnActivity].max(), prefix='act', columns=[columnActivity])
    
    return aggregated_data

# Example usage
columnCase = 'Case ID'  # Replace with the actual column name you want to group by
columnActivity = 'Activity'  # Replace with the actual column name you want to one-hot encode

result = agg_state_encoding(df_sorted, columnCase, columnActivity)

In [ ]:
result.value_counts()

### 2.2 Create Training and Held-out test data sets


Create a training and a held-out test data set. *Later in Task 3-4, the training data will be used to perform cross-validation. The held-out test data will be used to evaluate the performance of the selected models.*

Choose the size of your test data. Furthermore, how did you split the data? Motivate your choice when you discuss the experiment setup in your report. 



Tips: *You may consider reuse some of your code from Assignment 1 Task 1.2*

In [ ]:
# TODO: create training data and held-out test data for *data_Sepsis_ls*



# TODO: create training data and held-out test data for *data_Sepsis_ag*



## Task 3: Predicting Case Remaining Time - Regression Trees


In this task, you will use the regression tree (or random forest regression if you prefer) to learn a regression model to predict case remaining time. Very similar to how you have trained a classification model in Assignment 1, now perform the following steps to train a regression model. 

i) use the default values for the parameters to get a [Regression Tree](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html#sklearn.tree.DecisionTreeRegressor) (or a [Random Forest Regressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html)) running on the training data. (*Optional: visualize the tree, the feature importance, and compute the error measures to get an impression of the performance of the model*).

ii) use 5-fold cross-validation to determine a possibly better choice for the two parameters *min_samples_leaf* and *max_depth*
    
iii) create 2D or 3D plot that shows how the selected parameters affect the performance. 

iv) select the best-performing regression tree (or forest), i.e., the one that achieved the lowest cross-validated errors, and report all the error measures (MAE, MSE, RMSE, R^2) of the fitted model on the held-out test data. 

    
#### TIPS:
You may consider reuse the some of your code of Assignment 1 or use the [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) class (see an [example](https://www.dezyre.com/recipes/find-optimal-parameters-using-gridsearchcv-for-regression), but be aware that GridSearchSV does not return MAE or the other error measures (e.g., MSE, RMSE, R^2), you will need to update the scoring function)



In [ ]:
# TODO: import packages
from sklearn.tree import DecisionTreeRegressor
# from sklearn.ensemble import RandomForestRegressor

# TODO: using *data_Sepsis_ls* and test the default regression tree model (or the random forest regressor) 

# TODO: set the search space of the parameters *min_samples_leaf* and *max_depth*

# TODO: create 5-fold cross-validation

# TODO: learn an optimal regression tree model (random forest regressor)

# TODO: create 2D or 3D plot that shows how the selected parameter values affect the MAE (or RMSE). 

# TODO: compute the performance of the model on your held-out test data


In [ ]:
# TODO: repeat the above steps for *data_Sepsis_ag*



## Task 4. Predicting Case Remaining Time - kNN Regression


In this task, you will use the kNN Regression to learn a regression model to predict case remaining time. The same as task 3, now perform the following steps to train a regression model. 

i) use the default values for the parameters to get a [kNN Regression](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html#sklearn.neighbors.KNeighborsRegressor) running on the training data. (*Optional: compute the error measures to get an impression of the performance of the model).

ii) use 5-fold cross-validation to determine a possibly better choice for the two parameters *n_neighbors* and *weights* 
    
iii) create 2D or 3D plot that shows how the selected parameters affect the performance. 

iv) select the best-performing kNN, i.e., the one that achieved the lowest cross-validated errors, and report all the error measures (MAE, MSE, RMSE, R^2) of the fitted model on the held-out test data. 

    
#### TIPS:
The same here, you may consider reuse the some of your code of Assignment 1 or use the [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) class (see an [example](https://www.dezyre.com/recipes/find-optimal-parameters-using-gridsearchcv-for-regression), but be aware that GridSearchSV does not return MAE or the other error measures (e.g., MSE, RMSE, R^2), you will need to update the scoring function)







In [ ]:
# TODO: import packages
from sklearn.neighbors import KNeighborsRegressor


# TODO: using *data_Sepsis_ls* and test the default kNN Regressor model 

# TODO: set the search space of the parameters *n_neighbors* and *weights* 

# TODO: create 5-fold cross-validation

# TODO: learn an optimal kNN regressor

# TODO: create 2D or 3D plot that shows how the selected parameter values affect the MAE (or RMSE). 

# TODO: compute the performance of the model on your held-out test data

In [ ]:
# TODO: repeat the above steps for *data_Sepsis_ag*


## Task 5.  Report your results and discuss your findings

By now, you have applied two algorithms with different parameters on the two encodings of the data set. For each algorithm and each encoding, you have created tables or figures which you can add to your report. Discuss the results and their optimal performance. 

Create an overview table or figure that shows the optimal performance of each algorithm on the data set, for example, see the table here below. 


Discuss your findings and reflect on the following questions in your report:
- According to the error measures, which one would you suggest as the optimal model? 
- Are there any discrepancies between the MAE, MSE, RMSE, and R^2 measures in terms of which model performs the best? If yes, how would you explain these discrepancies. 
- Which one of the MAE, MSE, RMSE, and R^2 would you use for selecting the model? Why?
- Which one of the encoding would you suggest for this data set? Why?
- Which features have a big influence on predicting the remaining time?







| Encoding | Model | CV MAE  | Test MAE |  CV MSE  |  Test MSE  | CV R^2 | Test R^2 |... |
|------|------|------|------|------|------|------|------|-----|
|  Last-2-state | Regression Tree        |  |  | | | | |
|  Agg-state |  Regression Tree  |  |  | || | |
|   Last-2-state |kNN       |  |  | || | |
|   ... |...       |  |  | || | |











## Bonus Tasks 

We would like to challenge you with the following bonus tasks. For each task that is successfully completed, you may obtain max. 1 extra point. 

1. Implement or use another regression algorithm (for example, [Random Forest Regression](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html), [LinearRegresion](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html), [SVM Regression](https://scikit-learn.org/stable/auto_examples/svm/plot_svm_regression.html#sphx-glr-auto-examples-svm-plot-svm-regression-py)) or design your own algorithm that achieves a better MAE measure. Explain this in your report.
2. Implement techniques (e.g., preprocessing, feature engineering, feature selection, sampling) that help improve the MAE scores of existing models. For example, try out a feature selection for kNN or implement inter-case features. Explain this in your report.

